# 使用 PEFT 和 Adapter 微调模型
本示例展示如何使用 PEFT（参数高效微调）技术结合 Adapter 来微调模型。

In [ ]:
# 安装 peft 库（如果尚未安装）
!pip install peft

# 导入必要库
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig, get_peft_model, TaskType

# 加载基础模型和分词器
base_model_name = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=2)

# 配置 Adapter
peft_config = PeftConfig(
    task_type=TaskType.SEQ_CLS,  # 任务类型：文本分类
    inference_mode=False,       # 是否仅用于推理
    r=16,                       # 降低参数维度
    lora_alpha=32,              # LoRA 的 alpha 参数
    lora_dropout=0.1            # 随机失活比例
)

# 将 Adapter 添加到模型中
peft_model = get_peft_model(base_model, peft_config)

# 打印模型结构（包含 Adapter）
print(peft_model)

### 微调模型
以下是微调的简化代码。

In [ ]:
# 定义数据集（伪数据）
train_texts = ["这个商品非常好！", "我讨厌这家店。"]
train_labels = [1, 0]

# 编码输入数据
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
train_dataset = list(zip(train_encodings['input_ids'], train_labels))

# 定义训练参数
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./peft_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=10
)

# 创建 Trainer 并微调
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset
)

# 开始训练
trainer.train()